In [3]:
import pandas as pd

dat = pd.read_csv('data/SVD_sample_train.csv', names = ['user', 'item', 'rating'])
dat.head()

,user,item,rating
0,0,2209,1
1,0,2516,7
2,0,3638,2
3,0,5068,7
4,1,86,9


In [4]:
print(len(dat.user), len(dat.item))

94413 94413


In [5]:
dat.item.max()

5738

In [6]:
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csc, csc_matrix

#constants defining the dimensions of our User Rating Matrix (URM)
MAX_PID = 94413
MAX_UID = 94413

def readUrm():
    urm = np.zeros(shape=(MAX_UID,MAX_PID), dtype=np.float32)
    with open('data/SVD_sample_train.csv', 'r') as trainFile:
        urmReader = csv.reader(trainFile, delimiter=',')
        for row in urmReader:
            urm[int(row[1]), int(row[0])] = float(row[2])
    
    return csc_matrix(urm, dtype=np.float32)

In [7]:
def readUsersTest():
    uTest = dict()
    with open("data/SVD_sample_test.csv", 'r') as testFile:
        testReader = csv.reader(testFile, delimiter=',')
        for row in testReader:
            uTest[int(row[1])] = list()

    return uTest

In [8]:
def getMoviesSeen():
    moviesSeen = dict()
    with open("./trainSample.csv", 'r') as trainFile:
        urmReader = csv.reader(trainFile, delimiter=',')
        for row in urmReader:
            try:
                moviesSeen[int(row[0])].append(int(row[1]))
            except:
                moviesSeen[int(row[0])] = list()
                moviesSeen[int(row[0])].append(int(row[1]))

    return moviesSeen

In [9]:
import math as mt
import csv
from sparsesvd import sparsesvd

def computeSVD(urm, K):
    U, s, Vt = sparsesvd(urm, K)

    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i,i] = mt.sqrt(s[i])

    U = csr_matrix(np.transpose(U), dtype=np.float32)
    S = csr_matrix(S, dtype=np.float32)
    Vt = csr_matrix(Vt, dtype=np.float32)

    return U, S, Vt


In [10]:
from scipy.sparse.linalg import * #used for matrix multiplication

def computeEstimatedRatings(urm, U, S, Vt, uTest, K, test):
    rightTerm = S*Vt 

    estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
    for userTest in uTest:
        prod = U[userTest, :]*rightTerm

        #we convert the vector to dense format in order to get the indices of the movies with the best estimated ratings 
        estimatedRatings[userTest, :] = prod.todense()
        recom = (-estimatedRatings[userTest, :]).argsort()[:250]
        for r in recom:
            uTest[userTest].append(r)

            if len(uTest[userTest]) == 5:
                break

    return uTest, estimatedRatings

In [11]:
def calcrmsetest(arr):
        nusers = MAX_UID
        nmovies = MAX_PID
        sse = 0.0
        total = 0
        for i in range(nusers):
            for j in range(nmovies):
                if arr[i][j] == None: continue
                total += 1
                sse += (arr[i][j] - Ahat[i, j])**2
        return math.sqrt(sse/total)

def calcrmsetrain(self):
    nusers = self.nusers
    nmovies = self.nmovies
    sse = 0.0
    total = 0
    for i in range(nusers):
        for j in range(nmovies):
            total += 1
            sse += (self.M[i, j] - self.Ahat[i, j])**2
    return math.sqrt(sse/total)

In [12]:
def main():
    K = 90
    urm = readUrm()
    print(1)
    U, S, Vt = computeSVD(urm, K)
    print(2)
    uTest = readUsersTest()
    print(3)
#     moviesSeen = getMoviesSeen()
    uTest, estimatedRatings = computeEstimatedRatings(urm, U, S, Vt, uTest, K, True)
    print(4)
    rmse = calcrmsetest(uTest)
    return urm, U, S, Vt, uTest, rmse

In [13]:
urm, U, S, Vt, uTest, rmse = main()

1
2
3
4


NameError: name 'self' is not defined

IndexError: index (7000) out of range

ImportError: No module named 'numba'